In [1]:
import pandas as pd
import networkx as nx
import random
from scipy import special
from collections import defaultdict
import click
import os
from statsmodels.stats import multitest
import csv

In [2]:
"""
Constructs a NetworkX graph.

Input:
    - file : csv file

Output:
    - graph: NetworkX graph
"""
def construct_graph(file):
    df_ppi = pd.read_csv(file, sep = "\t", header=None, names=["Protein1", "interaction", "Protein2"])
    
    df_ppi.Protein1 = df_ppi['Protein1'].str.lower()
    df_ppi.Protein2 = df_ppi['Protein2'].str.lower()
    
    df_interactions = df_ppi.replace("in-complex-with", +1)
    df_interactions = df_interactions.replace("controls-expression-of", -1)
    df_interactions = df_interactions.replace("controls-state-change-of", -1)
    
    df_interactions['interaction'].loc[(df_interactions['interaction'] != +1) ] = 0
    df_interactions['interaction'].loc[(df_interactions['interaction'] != -1) ] = 0
    #& &&(df_interactions.loc[:,1] != -1)
    #df_dgxp_thr_filtered['fold-change'].loc[(df_dgxp_thr_filtered[DGXPCOLUMNS[1]] > 0)] = +1
    G = nx.DiGraph()
    
    for i in range(len(df_interactions)):
        prot1 = df_interactions.iloc[i,0]
        prot2 = df_interactions.iloc[i,2]
        #print(prot1, prot2)
        interaction = df_interactions.iloc[i,1]
        G.add_node(prot1)
        G.add_node(prot2)
        G.add_edge(prot1, prot2)
        G[prot1][prot2]['relation'] = interaction
 
        
    return G

In [3]:
path = "/Users/sophiakrix/Desktop/MechEnrichmentLab/NOTCH1_Intracellular.txt"
construct_graph(path)

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:

df = pd.read_csv(path, sep = "\t", header=None, names=["Protein1", "interaction", "Protein2"])

In [5]:
df.head()

,Protein1,interaction,Protein2
0,CCNC,in-complex-with,CDK8
1,CCNC,in-complex-with,CREBBP
2,CCNC,in-complex-with,EP300
3,CCNC,in-complex-with,KAT2A
4,CCNC,in-complex-with,KAT2B


In [6]:
df.columns

Index(['Protein1', 'interaction', 'Protein2'], dtype='object')

In [7]:
interactions_set = set(df.iloc[:,1])
interactions_set

{'controls-expression-of', 'controls-state-change-of', 'in-complex-with'}

In [8]:
interactions = df["interaction"].value_counts()
interactions

in-complex-with             234
controls-expression-of       72
controls-state-change-of     12
Name: interaction, dtype: int64

In [9]:
df_interactions = df.replace("in-complex-with", +1)
df_interactions = df_interactions.replace("controls-expression-of", -1)
df_interactions = df_interactions.replace("controls-state-change-of", -1)

In [10]:
df_interactions["interaction"].value_counts()

 1    234
-1     84
Name: interaction, dtype: int64

# NetworkX

In [11]:
G = construct_graph(path)

In [12]:
len(df_interactions)

318

In [13]:
for i in range(len(df_interactions)):
    prot1 = df_interactions.iloc[i,0]
    prot2 = df_interactions.iloc[i,2]
    interaction = df_interactions.iloc[i,1]
    G.add_node(prot1)
    G.add_node(prot2)
    G.add_edge(prot1, prot2)
    G[prot1][prot2]['relation'] = interaction

In [14]:
"""for n, nbrs in G.adj.items():
    for nbr, eattr in nbrs.items():
        relation = eattr['relation']
        #print(n, nbr, relation)"""

"for n, nbrs in G.adj.items():\n    for nbr, eattr in nbrs.items():\n        relation = eattr['relation']\n        #print(n, nbr, relation)"

In [15]:
#plt.subplot(121)
#nx.draw(G, with_labels=True, font_weight='bold')
#plt.subplot(122)
#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [16]:
# Nr edges and nr of nodes 

print("Number of edges: ",G.number_of_edges(),"\nNumber of nodes: ",G.number_of_nodes())


Number of edges:  580 
Number of nodes:  94


# Algorithmic Development

In [17]:
#G = nx.DiGraph()
LABEL = 'label'
df_dgxp = df_dgxp_pval_filtered
for i in range(len(df_interactions)):
    
    # add nodes to graph
    prot1 = df_interactions.loc[i, COLUMNS[0]]
    prot2 = df_interactions.loc[i, COLUMNS[2]]
    
    G.add_node(prot1)
    G.add_node(prot2)

    # add node attribute from dgxp file
    


#df = filter_dgxp(dgxp_file, sep, dgxp_columns, threshold)
df_ppi = df_interactions
gene_to_fc_dict = {}

    
for node in df_ppi.Protein1:
    if node in df_dgxp.index:
        #print(node)
        #fclabel = df_dgxp.loc[node, 'fold-change']
        #print(fclabel)
        gene_to_fc_dict[node] = {}
        gene_to_fc_dict[node]['label'] = df_dgxp.loc[node, 'fold-change']
        #print(f'Label for node {node} is now {fclabel}.')
        
        
    #print(gene_fc_dict)
   

NameError: name 'df_dgxp_pval_filtered' is not defined

In [21]:
"""
Randomly assigns labels of [-1,0,1] to nodes in a graph
Labels:
-1 : Downregulated
0 : No change
+1 : Upregulated

Input:
    - graph : the graph consisting of protein nodes 

Output:
    - prints list of nodes with associated attribute label
"""
def random_node_labels(graph):
    for node in graph.nodes():
        random_label = random.randint(-1,1)
        graph.nodes[node]['label'] = random_label
    print(graph.nodes.data())

In [22]:
#random_node_labels(G)

In [23]:
"""
Caclulates the  path between two nodes.

Input:
    - graph : NetworkX graph
    - source : upstream source node


Output:
    - dictionary of shortest path nodes between source node and all other nodes in graph
"""
def shortest_path(graph, source):
    for target in graph.nodes():
        shortest_paths = nx.shortest_path(graph, source)
    return shortest_paths

In [24]:
shortest_path(G, "ccnc")

{'ccnc': ['ccnc'],
 'cdk8': ['ccnc', 'cdk8'],
 'crebbp': ['ccnc', 'crebbp'],
 'ep300': ['ccnc', 'ep300'],
 'kat2a': ['ccnc', 'kat2a'],
 'kat2b': ['ccnc', 'kat2b'],
 'maml1': ['ccnc', 'maml1'],
 'maml2': ['ccnc', 'maml2'],
 'maml3': ['ccnc', 'maml3'],
 'mamld1': ['ccnc', 'mamld1'],
 'notch1': ['ccnc', 'notch1'],
 'rbpj': ['ccnc', 'rbpj'],
 'snw1': ['ccnc', 'snw1'],
 'hes1': ['ccnc', 'crebbp', 'hes1'],
 'hes5': ['ccnc', 'crebbp', 'hes5'],
 'hey1': ['ccnc', 'crebbp', 'hey1'],
 'hey2': ['ccnc', 'crebbp', 'hey2'],
 'heyl': ['ccnc', 'crebbp', 'heyl'],
 'myc': ['ccnc', 'crebbp', 'myc'],
 'rbx1': ['ccnc', 'notch1', 'rbx1'],
 'rps27a': ['ccnc', 'notch1', 'rps27a'],
 'skp1': ['ccnc', 'notch1', 'skp1'],
 'uba52': ['ccnc', 'notch1', 'uba52'],
 'ubb': ['ccnc', 'notch1', 'ubb'],
 'ubc': ['ccnc', 'notch1', 'ubc'],
 'tbl1xr1': ['ccnc', 'rbpj', 'tbl1xr1'],
 'tbl1x': ['ccnc', 'rbpj', 'tbl1x'],
 'tle1': ['ccnc', 'crebbp', 'hes1', 'tle1'],
 'tle2': ['ccnc', 'crebbp', 'hes1', 'tle2'],
 'tle3': ['ccnc', 'cr

In [25]:
"""
Check if node labels of source and target node are the same

Input:
    - graph: NetworkX graph
    - source: source upstream node

Output:
    - list of concordant and non-concordant nodes for the source node
"""


def count_concordance(graph, source):
    same_label = False

    nodes_dic = defaultdict(list)

    for target, path_nodes in shortest_path(graph, source).items():
        print(graph.nodes[source]['label'])
        # check if node labels of source and target are the same
        if graph.nodes[source]['label'] * graph.nodes[target]['label'] is 1:
            same_label = True

        # multiply the edge labels
        #edge_label = [G[path_nodes[i]][path_nodes[i + 1]]['relation'] * G[path_nodes[i]][path_nodes[i + 1]]['relation']
                      #for i in range(len(path_nodes) - 1)]

        edge_label = 1
        for i in range(len(path_nodes)-1):
            temp_edge_label = graph[path_nodes[i]][path_nodes[i+1]]['relation']
            edge_label *= temp_edge_label

        # concordant node
        if same_label == True and edge_label == +1:
            graph.nodes[target]['concordance'] = +1
            nodes_dic['concordant'].append(target)

        # non-concordant node
        if same_label == False and edge_label == -1:
            graph.nodes[target]['concordance'] = -1
            nodes_dic['non-concordant'].append(target)

        # no change node
        if graph.nodes[source]['label'] == 0 and graph.nodes[target]['label'] == 0:
            nodes_dic['no change'].append(target)

    return nodes_dic


In [26]:
G.nodes.data()

NodeDataView({'ccnc': {}, 'cdk8': {}, 'crebbp': {}, 'ep300': {}, 'kat2a': {}, 'kat2b': {}, 'maml1': {}, 'maml2': {}, 'maml3': {}, 'mamld1': {}, 'notch1': {}, 'rbpj': {}, 'snw1': {}, 'hes1': {}, 'hes5': {}, 'hey1': {}, 'hey2': {}, 'heyl': {}, 'myc': {}, 'cul1': {}, 'fbxw7': {}, 'rbx1': {}, 'skp1': {}, 'hdac10': {}, 'ncor1': {}, 'ncor2': {}, 'tbl1xr1': {}, 'tbl1x': {}, 'hdac11': {}, 'hdac1': {}, 'hdac2': {}, 'hdac3': {}, 'hdac4': {}, 'hdac5': {}, 'hdac6': {}, 'hdac7': {}, 'hdac8': {}, 'hdac9': {}, 'tle1': {}, 'tle2': {}, 'tle3': {}, 'tle4': {}, 'hif1a': {}, 'rps27a': {}, 'uba52': {}, 'ubb': {}, 'ubc': {}, 'CCNC': {}, 'CDK8': {}, 'CREBBP': {}, 'EP300': {}, 'KAT2A': {}, 'KAT2B': {}, 'MAML1': {}, 'MAML2': {}, 'MAML3': {}, 'MAMLD1': {}, 'NOTCH1': {}, 'RBPJ': {}, 'SNW1': {}, 'HES1': {}, 'HES5': {}, 'HEY1': {}, 'HEY2': {}, 'HEYL': {}, 'MYC': {}, 'CUL1': {}, 'FBXW7': {}, 'RBX1': {}, 'SKP1': {}, 'HDAC10': {}, 'NCOR1': {}, 'NCOR2': {}, 'TBL1XR1': {}, 'TBL1X': {}, 'HDAC11': {}, 'HDAC1': {}, 'HDAC2

In [27]:
count_concordance(G, "NOTCH1")

KeyError: 'label'

In [28]:

"""
Returns a dictionary of the nodes of the graph with their according
     - shortest path nodes
     - concordant nodes
     - non-concordant nodes
     - no change nodes

Input:
    - graph

Output:
    - dictionary of nodes 
"""
def nodes_dictionary(graph):
    dic = {}
    for node in graph.nodes():
        dic[node] = {}

        # concordant, non-concordant and no change nodes
        dic[node] = count_concordance(graph, node)

        # shortest path nodes
        dic[node]['shortest_path'] = list(shortest_path(graph, node).keys())

    return dic


In [29]:
nodes_dictionary(G)['CCNC']

KeyError: 'label'

In [30]:
"""
Calculates the concordance for an upstream node with its downstream nodes
Probability of getting at least the number of state changes consistent
with the direction
Input:
    - graph
    - p : probability of achieving a result

Output:
    - dictionary of p-values and corrected p-values for concordance
"""
def calculate_concordance(graph):
    concordance_dic = {}

    #assert 0 <= p and p <= 1, "p must be within [0,1]"

    for hyp_node in graph.nodes():
        
        if hyp_node not in graph.nodes():
            raise ValueError(f"The node {hyp_node} is not in the graph.")
        
        # n is number of trials
        n = len(shortest_path(graph, hyp_node).keys())
        # k is number of successful predictions
        k = len(count_concordance(graph, hyp_node)['concordant'])

        bin_coeff = special.binom(n, k)
        concordance = bin_coeff * (0.5 ** k) * (1 - 0.5) ** (n - k)
        concordance_dic[hyp_node] = {}
        concordance_dic[hyp_node]['p_val'] = concordance

    # correction for multiple testing
    pval_list = [concordance_dic[hyp_node]['p_val'] for hyp_node in graph.nodes()]
    reject, pvals_corrected = multitest.multipletests(pval_list,alpha=0.05,method='bonferroni')[:2]
    corrected_concordance_dic = {}
    for node, pval in zip(graph.nodes(),pvals_corrected):
        concordance_dic[node]['p_val_corrected'] = pval

    return concordance_dic

In [31]:
calculate_concordance(G)


KeyError: 'label'

In [32]:
"""
Writes the values for nodes, concordant_nodes, non_concordant_nodes, no_change_nodes, p_val, p_val_corrected to a csv file

Input:
- graph
- csv_output : path for output file 

Output:
- csv file
"""
def create_concordant_df(graph, csv_output):
    rows = []
    for node in graph.nodes():
        node_dict = {}
        node_dict['node'] = node
        node_dict['concordant_nodes'] = len(count_concordance(graph, node)['concordant'])
        node_dict['non_concordant_nodes'] = len(count_concordance(graph, node)['non-concordant'])
        node_dict['no_change'] = len(count_concordance(graph, node)['no change'])
        node_dict['p_val'] = calculate_concordance(graph)[node]['p_val']
        node_dict['p_val_corrected'] = calculate_concordance(graph)[node]['p_val_corrected']
        
        rows.append(node_dict)
        
    df_ = pd.DataFrame(rows)
    
    df_.to_csv(csv_output)
        

        
        

In [33]:
create_concordant_df(G, "/Users/sophiakrix/Desktop/Concordance_test.csv")

KeyError: 'label'

In [34]:
import os.path
import pandas as pd

print('Here:',os.path.abspath(os.path.dirname("__file__")))

try:
    dgxp_file = os.path.join(os.path.abspath(os.path.dirname("__file__")),'data', 'example_dgxp.txt')
except NameError:  # We are the main py2exe script, not a module
    import sys
    dgxp_file = os.path.join(os.path.abspath(os.path.dirname(sys.argv[0])),'data', 'example_dgxp.txt')

#dgxp_file = os.path.join(os.path.abspath(os.path.dirname(os.path.dirname(os.path.dirname(__file__)))),'data', 'example_dgxp.txt')
print(dgxp_file)
df = pd.read_csv(dgxp_file, sep='\t', index_col = 'Gene.symbol')
df.index = df.index.str.lower()

Here: /Users/sophiakrix/git/MechanismEnrichmentLab
/Users/sophiakrix/git/MechanismEnrichmentLab/data/example_dgxp.txt


In [35]:
df.head()

,ID,adj.P.Val,P.Value,t,B,logFC,Gene.title
Gene.symbol,,,,,,,
NaN,10415081,1.950000e-08,2.190000e-12,-28.259642,11.23514,-5.81675,NaN
NaN,10520121,1.950000e-08,2.190000e-12,-28.259642,11.23514,-5.81675,NaN
NaN,10578405,1.950000e-08,2.190000e-12,-28.259642,11.23514,-5.81675,NaN
NaN,10586076,1.950000e-08,2.190000e-12,-28.259642,11.23514,-5.81675,NaN
cers4,10569972,4.050000e-04,6.780000e-08,-11.595062,7.06249,-1.53425,ceramide synthase 4


In [36]:
df_dgxp = df[['logFC', 'adj.P.Val']].copy()

In [37]:
df_dgxp.columns = ["fold-change", "p-value"]

In [38]:
len(df_dgxp)

35556

In [39]:
df_dgxp.head()

,fold-change,p-value
Gene.symbol,,
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
cers4,-1.53425,4.050000e-04


In [40]:
df_dgxp = df_dgxp.dropna()

In [41]:
df_dgxp.head()

,fold-change,p-value
Gene.symbol,,
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
cers4,-1.53425,4.050000e-04


In [42]:
df_dgxp.index = df_dgxp.index.str.lower() 

In [43]:
df_dgxp.head()

,fold-change,p-value
Gene.symbol,,
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
cers4,-1.53425,4.050000e-04


In [44]:
#df_dgxp.loc[4,]

In [45]:
"""# alternative gene names

for gene in df_dgxp.loc[:, 'gene']:
    print(gene)
    print(df_dgxp.loc[gene, 'gene'])
    
    if '///' in df_dgxp.loc[gene, 'gene']:
        gene_names = gene.split('///')
        df_dgxp.gene.replace(gene, gene_names, in_place=True)
"""

"# alternative gene names\n\nfor gene in df_dgxp.loc[:, 'gene']:\n    print(gene)\n    print(df_dgxp.loc[gene, 'gene'])\n    \n    if '///' in df_dgxp.loc[gene, 'gene']:\n        gene_names = gene.split('///')\n        df_dgxp.gene.replace(gene, gene_names, in_place=True)\n"

In [49]:
df_dgxp.head()


,fold-change,p-value
Gene.symbol,,
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
cers4,-1.53425,4.050000e-04


In [50]:
DGXPCOLUMNS = ["gene", "fold-change", "p-value"]
threshold = 0.2

# filter dgxp

# filter nodes with p-value < 0.05
df_dgxp_pval_filtered = df_dgxp.loc[df_dgxp[DGXPCOLUMNS[2]] < 1.0]
print('pval filtered:',len(df_dgxp_pval_filtered))
# filter nodes with threshold (given by user)
df_dgxp_thr_filtered = df_dgxp_pval_filtered.loc[abs(df_dgxp_pval_filtered[DGXPCOLUMNS[1]]) > threshold]
print('thr filtered:',len(df_dgxp_thr_filtered))
# set fold change labels from float to +1 or -1
#df_dgxp_thr_filtered['fold-change'].loc[(df_dgxp_thr_filtered[DGXPCOLUMNS[1]] > 0)] = +1
#df_dgxp_thr_filtered['fold-change'].loc[(df_dgxp_thr_filtered[DGXPCOLUMNS[1]] < 0)] = -1

pval filtered: 35508
thr filtered: 12216


In [51]:
df_dgxp_thr_filtered.head()

,fold-change,p-value
Gene.symbol,,
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
NaN,-5.81675,1.950000e-08
cers4,-1.53425,4.050000e-04


In [65]:
df_dgxp_thr_filtered['FOLDCHANGE POSNEG'] = df_dgxp_thr_filtered['fold-change'].apply(lambda x: +1 if x > 0 else -1)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
(df_dgxp_thr_filtered['fold-change'] == 0).any()

False

In [59]:
df_dgxp_thr_filtered.head()

,fold-change,p-value,FOLD_CHANGE_POSNEG,FOLDCHANGE POSNEG
Gene.symbol,,,,
NaN,-5.81675,1.950000e-08,-1.0,0
NaN,-5.81675,1.950000e-08,-1.0,0
NaN,-5.81675,1.950000e-08,-1.0,0
NaN,-5.81675,1.950000e-08,-1.0,0
cers4,-1.53425,4.050000e-04,-1.0,0


In [ ]:
COLUMNS = ["Protein1", "interaction", "Protein2"]
DGXPCOLUMNS = ["fold-change", "p-value"]
GEO_COLUMNS = ['logFC', 'adj.P.Val']

ppi_file = os.path.join(os.path.abspath(os.path.dirname("__file__")),'data', 'example.txt')

df_ppi = pd.read_csv(ppi_file, sep='\t', header=None)
df_ppi.columns = ["Protein1", "interaction", "Protein2"]

df_ppi.Protein1 = df_ppi['Protein1'].str.lower()
df_ppi.Protein2 = df_ppi['Protein2'].str.lower()

df_interactions = df_ppi.replace("in-complex-with", +1)
df_interactions = df_interactions.replace("controls-expression-of", -1)
df_interactions = df_interactions.replace("controls-state-change-of", -1)
# replace all other interaction expressions with 0
df_interactions.loc[(df_interactions[COLUMNS[1]] != +1) & (df_interactions[COLUMNS[1]] != -1)] = 0


In [ ]:
set(df_interactions.Protein1.to_list())

In [ ]:
import pprint as pp
pp.pprint(gene_to_fc_dict)

In [19]:
all_tup = []



In [20]:
for target, path_nodes in shortest_path(graph, source).items():

    # check if node labels of source and target are the same
    if graph.nodes[source][LABEL] * graph.nodes[target][LABEL] is 1:
        same_label = True

NameError: name 'shortest_path' is not defined

In [ ]:
G.add_nodes_from(nodes_for_adding=gene_fc_dict) 

In [ ]:
G.nodes.data()

In [ ]:
 
    
            
    G.add_nodes_from(nodes_for_adding=gene_fc_dict, attr='label')    
    G.nodes.data()    
    """for node    
        G.add_edge(
            df_ppi.at[edge, 'Protein1'],
            df_ppi.at[edge, 'Protein2'],
            label=dgxp_dict[df_ppi.at[edge, 'interaction']]
        )"""

    """if prot1 in list(df_dgxp_thr_filtered['gene']):
        index = df_dgxp_thr_filtered[df_dgxp_thr_filtered.gene == prot1].index[0]
        G.nodes[prot1][LABEL] = df_dgxp_thr_filtered.loc[index, ['fold-change']]
        #print(df_dgxp.loc[index, 'fold-change'])
        print(f"Label for {prot1} is now {df_dgxp_thr_filtered.loc[index, 'fold-change']}")
       """ 
    
    # edge attributes
    interaction = df_interactions.loc[i, COLUMNS[1]]
    G.add_edge(prot1, prot2)
    G[prot1][prot2]['relation'] = interaction

In [ ]:
df_dgxp = df_dgxp_pval_filtered
df_dgxp.loc['ccnc', 'fold-change']

In [ ]:
#list(df_dgxp_thr_filtered['gene'])

In [ ]:
df_xp = df_dgxp_pval_filtered

In [ ]:
ppi_columns = 'Protein1,interaction,Protein2'
ppi_list = ppi_columns.split(',')
print(ppi_list)

In [ ]:
df1['warehouse'] = df1['store'].map(df2.set_index('store_code')['warehouse'])